In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout 
print('You are using TensorFlow version: ',tf.__version__)

In [ ]:
import numpy as np
import pandas as pd
import string
import os
import seaborn as sns
import matplotlib.pyplot as plt
import json

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from wordcloud import STOPWORDS

from sklearn.model_selection import train_test_split,KFold

import warnings
warnings.filterwarnings("ignore")

In [ ]:
prev_train_df = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')
test = pd.read_csv('../input/jigsaw-toxic-severity-rating/validation_data.csv')
sample = pd.read_csv('../input/jigsaw-toxic-severity-rating/sample_submission.csv')
target = pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')

In [ ]:
test=pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')

In [ ]:
test

In [ ]:
prev_train_df["severe_toxic"] = prev_train_df["severe_toxic"] * 2.5
prev_train_df["threat"] = prev_train_df["threat"] *3
prev_train_df["identity_hate"] = prev_train_df["identity_hate"] * 2

prev_train_df["total_toxic"] = prev_train_df[["toxic", "severe_toxic", "obscene", "threat", "insult",
                                              "identity_hate"]].sum(axis = 1).astype(np.int64)

In [ ]:
train_df = prev_train_df[["comment_text", "total_toxic"]]

In [ ]:
train_df["Comment"] = train_df['comment_text'].str.replace('[^\w\s\n]','')
train_df.head()

In [ ]:
train_df.drop(['comment_text'],axis=1)

In [ ]:
text=train_df.loc[1:50000,['Comment']]
target=train_df.loc[1:50000,['total_toxic']]

In [ ]:
text.shape

In [ ]:
train_df['text'] = train_df['Comment'].apply(lambda x:x.lower())

In [ ]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

In [ ]:
#removing the stop words from the corpus
train_df['filter1'] =train_df['text'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_words)]))

In [ ]:
train_df['filter1']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfvec = TfidfVectorizer(analyzer = 'char_wb', ngram_range = (3,5))
tfv = tfvec.fit_transform(train_df["filter1"])

In [ ]:
X=tfv[1:50001]
Y=train_df.loc[1:50000,['total_toxic']]

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size = 0.2,random_state = 0)

In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(x_train, y_train)

In [ ]:
tfv_comments = tfvec.transform(test["text"])
pred = regressor.predict(tfv_comments)

In [ ]:
regressor.score(x_test,y_test)

In [ ]:
sub = pd.DataFrame()
sub["comment_id"] = test["comment_id"]
sub["score"] = pred
sub.to_csv('submission.csv',index=False)
sub